# config

> description: Configuring nbdev and bootstrapping notebook export

In [ ]:
#|hide
#|default_exp config

In [ ]:
#|export
from datetime import datetime
from fastcore.imports import *
from fastcore.foundation import *
from fastcore.basics import *
from fastcore.imports import *
from fastcore.meta import *
from fastcore.script import *
from fastcore.xdg import *
from fastcore.xtras import *

import ast,functools
from IPython.display import Markdown
from configparser import ConfigParser
from execnb.nbio import read_nb,NbCell
from pprint import pformat,pprint

In [ ]:
#|hide
from fastcore.test import *
import tempfile

## Configuring nbdev

nbdev is heavily customizeable, thanks to the configuration system defined in this module. There are 2 ways to interact with nbdev's config:

- **In the terminal:** `nbdev_create_config` creates a config file (if you're starting a new project use `nbdev_new` instead)
- **In your library:** `get_config` returns a [`Config`](https://fastcore.fast.ai/foundation.html#config) object.

Read on for more about how these work.

### Creating a config file -

In [ ]:
#|export
_nbdev_home_dir = 'nbdev' # sub-directory of xdg base dir
_nbdev_config_name = 'settings.ini'

In [ ]:
#|exporti
def apply_defaults(
    cfg,
    lib_name:str=None, # Package name, defaults to local repo folder name
    branch='master', # Repo default branch
    git_url='https://github.com/%(user)s/%(lib_name)s', # Repo URL
    custom_sidebar:bool_arg=False, # Use a custom sidebar.yml?
    nbs_path='.', # Path to notebooks
    lib_path='%(lib_name)s', # Path to package root
    doc_path='_docs', # Path to rendered docs
    tst_flags='', # Test flags
    version='0.0.1', # Version of this release
    doc_host='https://%(user)s.github.io',  # Hostname for docs
    doc_baseurl='/%(lib_name)s',  # Base URL for docs
    keywords='nbdev jupyter notebook python', # Package keywords
    license='apache2', # License for the package
    copyright:str=None, # Copyright for the package, defaults to '`current_year` onwards, `author`'
    status='3', # Development status PyPI classifier
    min_python='3.7', # Minimum Python version PyPI classifier
    audience='Developers', # Intended audience PyPI classifier
    language='English', # Language PyPI classifier
    recursive:bool_arg=False, # Include subfolders in notebook globs?
    black_formatting:bool_arg=False, # Format libraries with black?
    readme_nb='index.ipynb', # Notebook to export as repo readme
    title='%(lib_name)s', # Quarto website title
    allowed_metadata_keys='', # Preserve the list of keys in the main notebook metadata
    allowed_cell_metadata_keys='', # Preserve the list of keys in cell level metadata
    jupyter_hooks=True, # Run Jupyter hooks?
    clean_ids=True, # Remove ids from plaintext reprs?
    custom_quarto_yml=False, # Use a custom _quarto.yml?
):
    "Apply default settings where missing in `cfg`."
    if lib_name is None:
        _parent = Path.cwd().parent
        lib_name = _parent.parent.name if _parent.name=='nbs' else _parent.name
    if copyright is None and hasattr(cfg,'author'): copyright = f"{datetime.now().year} ownwards, {cfg.author}"
    for k,v in locals().items():
        if not (k.startswith('_') or k=='cfg' or k in cfg): cfg[k] = v
    return cfg

In [ ]:
#|export
@call_parse
@delegates(apply_defaults, but='cfg')
def nbdev_create_config(
    user:str, # Repo username
    author:str, # Package author's name
    author_email:str, # Package author's email address
    description:str, # Short summary of the package
    path:str='.', # Path to create config file
    cfg_name:str=_nbdev_config_name, # Name of config file to create
    **kwargs
):
    "Create a config file. Settings can be passed as command-line arguments."
    d = {k:v for k,v in locals().items() if k not in ('path','cfg_name','kwargs')}
    cfg = Config(path, cfg_name, d, save=False)
    cfg = apply_defaults(cfg, **kwargs)
    cfg.save()

The table above also serves as a full reference of nbdev's settings (excluding the `path` and `cfg_name` parameters which decide where the config file is saved). For more about PyPI classifiers, see [_Classifiers_](https://pypi.org/classifiers/).

You can create a config file by passing all of the required settings via the command line (as well as any optional settings you'd like to override):

In [ ]:
%%sh
nbdev_create_config fastai fastai info@fast.ai 'A test project' --path ../tests

Here's what the first few lines look like:

In [ ]:
%%sh
head -n5 ../tests/settings.ini

[DEFAULT]
user = fastai
author = fastai
author_email = info@fast.ai
description = A test project


### Reading a config file -

In [ ]:
#|export
def _nbdev_config_file(cfg_name=_nbdev_config_name, path=None):
    cfg_path = path = Path.cwd() if path is None else Path(path)
    while cfg_path != cfg_path.parent and not (cfg_path/cfg_name).exists(): cfg_path = cfg_path.parent
    if not (cfg_path/cfg_name).exists(): cfg_path = path
    return cfg_path/cfg_name

In [ ]:
#|hide
test_eq(_nbdev_config_file(), Path.cwd().parent/'settings.ini')

In [ ]:
#|export
def _xdg_config_paths(cfg_name=_nbdev_config_name):
    xdg_config_paths = reversed([xdg_config_home()]+xdg_config_dirs())
    return [o/_nbdev_home_dir/cfg_name for o in xdg_config_paths]

In [ ]:
#|export
@functools.lru_cache(maxsize=None)
def get_config(cfg_name=_nbdev_config_name, path=None):
    """Return nbdev config.\n\nSearches up from `path` until `cfg_name` is found. User settings are loaded from
`~/.config/nbdev/{cfg_name}`. Unspecified optional settings return defaults."""
    cfg_file = _nbdev_config_file(cfg_name, path)
    extra_files = _xdg_config_paths(cfg_name)
    cfg = Config(cfg_file.parent, cfg_file.name, extra_files=extra_files)
    return apply_defaults(cfg)

See `nbdev_create_config` for a full reference of nbdev's settings.

In [ ]:
#|hide
try: _get_config,get_config = get_config,get_config.__wrapped__ # Bypass cache during dev and tests
except AttributeError: pass

Let's test it out in nbdev's own repo:

In [ ]:
cfg = get_config()

`cfg` is a fastcore [`Config`](https://fastcore.fast.ai/foundation.html#config) object, so you can access keys as attributes:

`get_config` searches for repo `settings.ini` in the current directory, and then in all parent directories, stopping when it is found. Default values for optional settings are applied to the resulting `Config`, see `nbdev_create_config` for a full reference of nbdev's settings.

In [ ]:
p = Path.cwd().parent
test_eq(cfg.lib_name, 'nbdev')
test_eq(cfg.git_url, 'https://github.com/fastai/nbdev')

Its own path and parent are attributes too:

In [ ]:
test_eq(cfg.config_path, p)
test_eq(cfg.config_file, p/'settings.ini')

For convenience, use `path` to resolve settings that specify project-relative paths:

In [ ]:
test_eq(cfg.path('doc_path'), p/'_docs')
test_eq(cfg.path('lib_path'), p/'nbdev')
test_eq(cfg.path('nbs_path'), p/'nbs')

It automatically returns defaults for keys not specified in the config file. Let's create an empty config file to demonstrate:

In [ ]:
#|hide
!rm ../tests/test_settings.ini 2>/dev/null

In [ ]:
Config('../tests/', 'test_settings.ini', {'repo': 'my-project', 'author': 'fastai'});

In [ ]:
%%sh
cat ../tests/test_settings.ini

[DEFAULT]
repo = my-project
author = fastai



In [ ]:
cfg = get_config('test_settings.ini', '../tests/')
test_eq(cfg.repo, 'my-project')
test_eq(cfg.lib_path, 'nbdev')
test_eq(cfg.copyright, '2022 ownwards, fastai')

In fact, you can return a default config even if you don't have a settings file. This is to support certain nbdev commands work outside of nbdev repos:

In [ ]:
%%sh
rm ../tests/test_settings.ini

In [ ]:
cfg = get_config('test_settings.ini', '../tests')
test_eq(cfg.lib_path, 'nbdev')
test_eq(cfg.nbs_path, '.')

You can customise nbdev for all repositories for your user with a ~/.config/nbdev/settings.ini file (by default, although we follow the broader [XDG specification](https://specifications.freedesktop.org/basedir-spec/basedir-spec-latest.html)).

In [ ]:
%%sh
mkdir ../tests/user_settings

For example, you could globally disable nbdev's Jupyter hooks by creating the following user settings file (we override `XDG_CONFIG_DIR` to demonstrate):

In [ ]:
#|hide
from contextlib import contextmanager

In [ ]:
#|hide
@contextmanager
def env(k, v):
    orig = os.environ.get(k,None)
    try:
        os.environ[k] = str(v)
        yield
    finally:
        if orig is None: del os.environ[k]
        else: os.environ[k] = orig

In [ ]:
Config('../tests/user_settings', 'test_settings.ini', {'jupyter_hooks': False})
with env('XDG_CONFIG_DIR', Path('../tests/user_settings').resolve()):
    cfg = get_config('test_settings.ini', '../tests')

test_eq(str2bool(cfg.jupyter_hooks), True)

In [ ]:
#|hide
!rm -r ../tests/user_settings 2>/dev/null

In [ ]:
#|export
def config_key(c, default=None, path=True, missing_ok=None):
    "Deprecated: use `get_config().get` or `get_config().path` instead."
    warn("`config_key` is deprecated. Use `get_config().get` or `get_config().path` instead.", DeprecationWarning)
    return get_config().path(c, default) if path else get_config().get(c, default)

## Helpers

In [ ]:
#|export
def create_output(txt, mime):
    "Add a cell output containing `txt` of the `mime` text MIME sub-type"
    return [{"data": { f"text/{mime}": str(txt).splitlines(True) },
             "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]

In [ ]:
#|export
def show_src(src, lang='python'): return Markdown(f'```{lang}\n{src}\n```')

In [ ]:
show_src("print(create_output('text', 'text/plain'))")

```python
print(create_output('text', 'text/plain'))
```

##  Exporting a basic module

In [ ]:
#|export
_re_version = re.compile('^__version__\s*=.*$', re.MULTILINE)
_init = '__init__.py'

def update_version(path=None):
    "Add or update `__version__` in the main `__init__.py` of the library."
    path = Path(path or get_config().path("lib_path"))
    fname = path/_init
    if not fname.exists(): fname.touch()
    version = f'__version__ = "{get_config().version}"'
    code = fname.read_text()
    if _re_version.search(code) is None: code = version + "\n" + code
    else: code = _re_version.sub(version, code)
    fname.write_text(code)

def _has_py(fs): return any(1 for f in fs if f.endswith('.py'))

def add_init(path=None):
    "Add `__init__.py` in all subdirs of `path` containing python files if it's not there already."
    # we add the lowest-level `__init__.py` files first, which ensures _has_py succeeds for parent modules
    path = Path(path or get_config().path("lib_path"))
    path.mkdir(exist_ok=True)
    if not (path/_init).exists(): (path/_init).touch()
    for r,ds,fs in os.walk(path, topdown=False):
        r = Path(r)
        subds = (os.listdir(r/d) for d in ds)
        if _has_py(fs) or any(filter(_has_py, subds)) and not (r/_init).exists(): (r/_init).touch()
    update_version(path)

Python modules require a `__init.py__` file in all directories that are modules. We assume that all directories containing a python file (including in subdirectories of any depth) is a module, and therefore add a `__init__.py` to each.

In [ ]:
with tempfile.TemporaryDirectory() as d:
    d = Path(d)
    (d/'a/b').mkdir(parents=True)
    (d/'a/b/f.py').touch()
    (d/'a/c').mkdir()
    add_init(d)
    assert not (d/'a/c'/_init).exists(), "Should not add init to dir without py file"
    for e in [d, d/'a', d/'a/b']: assert (e/_init).exists(),f"Missing init in {e}"

## Export -

In [ ]:
#|export
def write_cells(cells, hdr, file, offset=0):
    "Write `cells` to `file` along with header `hdr` starting at index `offset` (mainly for nbdev internal use)."
    for cell in cells:
        if cell.source.strip(): file.write(f'\n\n{hdr} {cell.idx_+offset}\n{cell.source}')

In [ ]:
#|export
def basic_export_nb(fname, name, dest=None):
    "Basic exporter to bootstrap nbdev."
    if dest is None: dest = get_config().path('lib_path')
    add_init()
    fname,dest = Path(fname),Path(dest)
    nb = read_nb(fname)

    # grab the source from all the cells that have an `export` comment
    cells = L(cell for cell in nb.cells if re.match(r'#\s*\|export', cell.source))

    # find all the exported functions, to create `__all__`:
    trees = cells.map(NbCell.parsed_).concat()
    funcs = trees.filter(risinstance((ast.FunctionDef,ast.ClassDef))).attrgot('name')
    exp_funcs = [f for f in funcs if f[0]!='_']

    # write out the file
    with (dest/name).open('w') as f:
        f.write(f"# %% auto 0\n__all__ = {exp_funcs}")
        write_cells(cells, f"# %% {fname.relpath(dest)}", f)
        f.write('\n')

This is a simple exporter with just enough functionality to correctly export this notebook, in order to bootstrap the creation of nbdev itself.

In [ ]:
#|hide
#| eval: false
path = Path('../nbdev')
(path/'read.py').unlink(missing_ok=True)

basic_export_nb("01_read.ipynb", 'read.py')

g = exec_new('from nbdev import read')
assert g['read'].add_init
assert 'add_init' in g['read'].__all__